<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/Summorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bert-extractive-summarizer
!pip install -U python-docx
!pip install -U transformers
#!pip install -U textract
#!apt-get install antiword

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 890kB 19.4MB/s 
     |████████████████████████████████| 3.2MB 21.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=990605bbd6668e4cacafc65544a968aa66cae5708cd95418e7e8c1025509b1da
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 5.5MB 5.2MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=0e3495fe10aecc24327f92d37ede0a7f0954067dc66575a0e0a5dd66e707847f
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.3.0)


In [9]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-02-09 05:03:16--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.242, 5.45.205.243, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm909.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-02-09 05:03:16--  http://cache-mskm909.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm909.cdn.yandex.net (cache-mskm909.cdn.yandex.net)... 5.45.220.19, 2a02:6b8:0:2002::1a
Connecting to cache-mskm909.cdn.yandex.net (cache-mskm909.cdn.yandex.net)|5.45.220.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [3]:
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkrText.csv?token=AITWEHZ4A4DHCMHXSKF2FOTAFIAJC

--2021-02-09 05:02:40--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30447 (30K) [text/plain]
Saving to: ‘people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA’

people.csv?token=AI 100%[===================>]  29.73K  --.-KB/s    in 0.002s  

2021-02-09 05:02:40 (11.7 MB/s) - ‘people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA’ saved [30447/30447]

--2021-02-09 05:02:40--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

In [4]:
import requests
import pandas as pd
from docx import Document
from summarizer import Summarizer
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import string

from pymystem3 import Mystem
import re
import math

#import textract

nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
russian_stopwords.append('рис')
russian_stopwords.append('т.к.')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
vkr = pd.read_csv('/content/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE')
vkrText = pd.read_csv('/content/vkrText.csv?token=AITWEHZ4A4DHCMHXSKF2FOTAFIAJC')
vkrText.head()

,Unnamed: 0,Unnamed: 0.1,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file,fullText,shortText
0,0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,NaN,NaN,NaN
1,1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...,\n ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕНН...,NaN
2,2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
3,3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
4,4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN


In [ ]:
vkrText = vkr.copy()
vkrText['fullText'] = ''
vkrText['shortText'] = ''

In [ ]:
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        try:
            open('test.doc', 'wb').write(r.content)
            f = open('test.doc', 'rb')
            document = Document(f)
            f.close()
            successful.append(url)
        except:
            unsuccessful.append(url)
            vkrText.loc[row, 'fullText' ] = ''
            vkrText.loc[row, 'shortText'] = ''
            continue
        
        fullText = ""
        for para in document.paragraphs:
            fullText+=(para.text)+" "
        vkrText.loc[row, 'fullText' ] = fullText
        vkrText.loc[row, 'shortText'] = ''
    else:
        vkrText.loc[row, 'link_file'] = ''
        vkrText.loc[row, 'fullText' ] = ''
        vkrText.loc[row, 'shortText'] = ''

100%|██████████| 1708/1708 [03:41<00:00,  7.70it/s]


In [ ]:
len(successful), len(unsuccessful)

(0, 72)

In [ ]:
vkrText.to_csv('vkrText.csv')

In [ ]:
tmp = vkrText[vkrText['link_file']!='']
tmp[tmp['fullText']=='']
tmp

In [ ]:
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        if (vkrText.loc[row,'fullText'])!='':
            continue
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        try:
            open('test.doc', 'wb').write(r.content)
            vkrText.loc[row, 'fullText' ] = textract.process("/content/test.doc").decode("utf-8")
            successful.append(url)
        except:
            unsuccessful.append(url)
            continue

100%|██████████| 1708/1708 [04:33<00:00,  6.24it/s]


In [ ]:
len(successful), len(unsuccessful)

(22, 50)

In [ ]:
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        if (vkrText.loc[row,'fullText'])!='':
            continue
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        open('test.pdf', 'wb').write(r.content)
        try:
            vkrText.loc[row, 'fullText'] = textract.process("/content/test.pdf").decode("utf-8")
        except:
            unsuccessful.append(url)

In [ ]:
vkrText.to_csv('vkrText.csv')

In [ ]:
 len(unsuccessful), unsuccessful

(20,
 ['http://lms.hse.ru/ap_service.php?getwork=1&guid=25FDAA4B-6E10-4D95-9B41-EB728C9F1D56',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=645FE2D8-35C1-420D-9F27-92EDDE1F7B21',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=2B42CEB5-D802-48FB-BF2B-0C961D408E22',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=3D0A5337-5604-4646-A4E2-3D9A63A3160A',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=880D0A43-B9A7-47EE-9440-E200964C399D',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=DE0CC8F2-DC9B-407F-AE0B-3579C1C900D2',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=819D0194-4397-400A-945C-05E32B4AED65',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=DFDFD1A3-A42A-4F71-AE09-A4CFA04CE491',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=84D8F5CC-3FAA-4BC0-BBCD-B9050161E620',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=420F4757-25A0-4B60-9EC9-2CDB58EF5E45',
  'http://lms.hse.ru/ap_service.php?getwork=1&guid=100D11A6-E89B-45B0-B2B7-44592CE433EC',
  'ht

In [16]:
custom_config = AutoConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)

In [13]:
def lowerCase(Corpus):
    corpus = "".join([i.lower().replace('\n',' ') for i in Corpus])
    return corpus

PUNCT_TO_REMOVE = "'#$%&\()*+-/<=>|"
def remove_punctuation(Corpus):
    """custom function to remove the punctuation"""
    return "".join([i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) for i in Corpus])

def remove_stopwords(Corpus):
    """custom function to remove the stopwords"""
    return " ".join([token for token in Corpus.split() if token not in russian_stopwords])

def remove_freqwords(Corpus):
    cnt = Counter()
    for text in Corpus:
        for word in text.split():
            cnt[word] += 1
    FREQWORDS = set([w for (w, wc) in cnt.most_common(30)])
    return " ".join([word for word in str(Corpus).split() if word not in FREQWORDS])

def remove_urls(Corpus):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    return_text=[]
    for text in Corpus:
        text=url_pattern.sub(r'', text)
        text=html_pattern.sub(r'',text)
        return_text.append(text)
    return "".join(i for i in return_text)
    
def remove_numbers(Corpus):
    text = ["".join(i for i in text if not i.isdigit()) for text in Corpus]
    return "".join(i for i in text)

def Stem(Corpus):
    stemmer = SnowballStemmer("russian") 
    return "".join(m.lemmatize(Corpus))

def Lemm(Corpus):
    m = Mystem()
    return "".join(m.lemmatize(Corpus))

def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 

def clearTextAfterGarbage(Corpus):
    corpus = Corpus
    corpus = lowerCase(Corpus)
    corpus = remove_urls(corpus)
    corpus = remove_numbers(corpus)
    corpus = remove_stopwords(corpus)
    corpus = remove_punctuation(corpus)
    corpus = Lemm(corpus)
    #corpus = Stem(corpus)
    corpus = remove_freqwords(corpus)
    return corpus

In [14]:
text = clearTextAfterGarbage(vkrText.loc[1, 'fullText'])
text

'пермский филиал федеральный государственный автономный образовательный учреждение высокий образование «национальный исследовательский университет «высокий школа экономика» факультет экономика, менеджмент бизнесинформатика стерледев наталья владимировна оценка эффективность инвестиционный проект выпускной квалификационный работа бакалаврский работа студент образовательный программа бакалавриат «экономика» направление подготовка .. экономика рецензент руководитель а.б. прасол а.с. абашев пермь, год аннотация цель работа являться оценка эффективность инвестиционный проект аквацентр. поставлять задача: описывать принцип организация предприятие перечень предоставлять услуга, проанализировать рынок, спрос конкурент, рассчитать сумма первоначальный вложение, разрабатывать бюджет доход расход, бюджет движение денежный средство, график финансирование, рассчитать показатель эффективность проект. итог получать следующий результат: исследовать проект аквацентр «лагуна» первый год реализация генер

In [18]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = modelSum(text, min_length=20, max_length=100)
summary = "".join(result)
print(len(summary), summary)

4165 irr проект превышать средневзвешенный стоимость капитал wacc, это говорить эффективность проект. целое, проект являться эффективный финансовый реализовать. оценка эффективность проект…………………………...……. .. оценка риск метод анализ чувствительность ……….… цель работа являться оценка эффективность инвестиционный проект. приложение вынести расчетный таблица. следующий блок, который предлагать рассматривать остервальдера «основной направление деятельность». длительность шаг расчет измеряться год доля год. необходимо, шаг соответствовать продолжительность важный фаза жизненный цикл проект. срок окупаемость — показатель индивидуальный. pi profitability index индекс отдача инвестиция. метод дельфи – метод сбор информация, использовать достижение консенсус эксперт какомулибо вопрос. получать, число, экспертный оценка. часть становиться постоянный клиент, это гарантировать успех. здание санузел, два душевой мужской женский два минисауна, два комната переодевание. также комната персонал, кафе к

In [19]:
%%time
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
for row in tqdm(range(vkrText.shape[0])):
    if type(vkrText.loc[row, 'fullText'])==str:
        #print(row, vkrText.loc[row, 'fullText'])
        text = clearTextAfterGarbage(vkrText.loc[row, 'fullText'])
        result = modelSum(text, min_length=20, max_length=100)
        summary = "".join(result)
        vkrText.loc[row, 'shortText'] = summary

100%|██████████| 1708/1708 [4:13:35<00:00,  8.91s/it]

CPU times: user 4h 1min 43s, sys: 16min 41s, total: 4h 18min 24s
Wall time: 4h 13min 35s


In [21]:
vkrText['shortText'].nunique()

481

In [22]:
vkrText.head()

,Unnamed: 0,Unnamed: 0.1,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file,fullText,shortText
0,0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,NaN,NaN,NaN
1,1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...,\n ПЕРМСКИЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО ГОСУДАРСТВЕНН...,irr проект превышать средневзвешенный стоимост...
2,2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
3,3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN
4,4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,NaN,NaN


In [26]:
vkrText.drop(['Unnamed: 0','Unnamed: 0.1','fullText'], axis=1, inplace=True)

In [27]:
vkrText.to_csv('vkrTextShort.csv')